# Example of How to Use GenAI with ConvoKit genai Module

The ConvoKit GenAI module provides a unified interface for working with large language models (LLMs) while doing conversational analysis in ConvoKit. It supports multiple providers including OpenAI GPT, Google Gemini, and local models through a simple factory pattern. This module makes it easy to integrate AI-powered text generation into your ConvoKit workflows for diverse tasks. The module handles API key management, response formatting, and provides consistent interfaces across different LLM providers.


## Setup config for GenAI with GPT

Setting up config info to access models is mandatory but simple. For models we implemented (GPT and Gemini), we provide methods to set API keys so they are stored in the environment. For other models or local LLMs, users can also implement them in similar manner. Here we provide a simple demonstration with configuring for OpenAI's GPT model.

In [23]:
from convokit.genai.genai_config import GenAIConfigManager

In [ ]:
config = GenAIConfigManager()
config.set_api_key("gpt", "YOUR API KEY")
print(f"Successfully set OpenAI API key in config.")

Successfully set OpenAI API key in config.


## Initialize clients to Communicate with models

After setting the API key, we are ready to communicate with the models. Retrieve response the same as you would normally do interacting with models through API. However, we do wrap the LLM responses in a unified class, so we can handle all LLM response format easily. Users are expected to follow similar template when implementing clients for other models.

In [25]:
from convokit.genai import get_llm_client

MODEL_PROVIDER = "gpt"
client = get_llm_client(MODEL_PROVIDER, config)

In [26]:
response = client.generate([{"role": "user", "content": "Tell me a fun fact about Cornell University."}])
print("Text:", response.text)
print("Tokens:", response.tokens)
print("Latency:", response.latency)

Text: A fun fact about Cornell University is that it is home to the world's first university-based hotel management program. Established in 1922, the Cornell School of Hotel Administration has become a leader in hospitality education, attracting students from around the globe who aspire to careers in the hospitality industry. The program is renowned for its rigorous curriculum and strong connections to the industry, making it a top choice for aspiring hotel and restaurant managers.
Tokens: 99
Latency: 1.7928547859191895


# Setup ConvoKit GenAI with Google Gemini Through Vertex AI

Very similar to GPT. Checkout Vertex AI: https://cloud.google.com/vertex-ai/generative-ai/docs/start/quickstarts/quickstart-multimodal

In [ ]:
MODEL_PROVIDER = "gemini"
MODEL = "gemini-2.0-flash-001"
config.set_google_cloud_config("YOUR PROJECT", "YOUR LOCATION")

In [28]:
client = get_llm_client(MODEL_PROVIDER, config)
response = client.generate("Tell me a fun fact about Cornell University.")
print("Text:", response.text)

Text: Here's a fun fact about Cornell University:

Cornell has a tradition called "Dragon Day" where architecture students build a giant dragon and parade it across campus. The dragon is often pitted against a phoenix built by engineering students, leading to a playful rivalry and a spectacle of creativity and engineering!



# Using LLMPromptTransformer with ConvoKit

The LLMPromptTransformer provides a powerful way to apply LLM processing to different levels of ConvoKit objects (utterances, conversations, speakers, or corpus). Let's demonstrate this with the friends corpus.


In [ ]:
from convokit import Corpus, download
from convokit.genai import LLMPromptTransformer

corpus = Corpus(filename=download("friends-corpus"))

first_convo = corpus.get_conversation(corpus.get_conversation_ids()[0])
assert len(first_convo.get_utterance_ids()) > 2
utterances = first_convo.get_utterance_ids()[:2]

print(f"First conversation ID: {first_convo.id}")
print(f"We'll process the first 2 utterances:")
for i, uttid in enumerate(utterances):
    utt = corpus.get_utterance(uttid)
    print(f"  Utterance {i+1}: {utt.speaker.id}: {utt.text[:100]}...")


Dataset already exists at /reef/kz88/convokit/download_corpus/friends-corpus
First conversation ID: s01_e01_c01_u001
We'll process the first 2 utterances:
  Utterance 1: Monica Geller: There's nothing to tell! He's just some guy I work with!...
  Utterance 2: Joey Tribbiani: C'mon, you're going out with the guy! There's gotta be something wrong with him!...


## Example: Sentiment Analysis on Utterances

Let's create a GenAI transformer that analyzes the sentiment of utterances and stores the result as metadata.


In [ ]:
sentiment_transformer = LLMPromptTransformer(
    provider="gpt",
    model="gpt-4o-mini",
    object_level="utterance",  # Process at utterance level
    prompt="Analyze the sentiment of the following text and respond with just one word: 'positive', 'negative', or 'neutral'. Text: {formatted_object}",
    formatter=lambda utterance: utterance.text,
    metadata_name="gpt_sentiment",  # Store result in 'gpt_sentiment' metadata field
    selector=lambda utterance: utterance.id in utterances,
    config_manager=config
)

corpus = sentiment_transformer.transform(corpus)

print("Sentiment analysis results:")
for uttid in utterances:
    utt = corpus.get_utterance(uttid)
    sentiment = utt.meta.get("gpt_sentiment", "Not processed")
    print(f"  {utt.speaker.id}: '{utt.text[:50]}...' -> Sentiment: {sentiment}")


Sentiment analysis results:
  Monica Geller: 'There's nothing to tell! He's just some guy I work...' -> Sentiment: Negative
  Joey Tribbiani: 'C'mon, you're going out with the guy! There's gott...' -> Sentiment: Negative


## Testing Prompts on Single Objects

The `transform_single` method allows you to test your prompts on individual objects without processing an entire corpus. This function allows user to test prompt development.


In [ ]:
# Test with a single utterance using string input
test_text = "I absolutely love this new feature! It's amazing!"
result = sentiment_transformer.transform_single(test_text)
print(f"Input: {test_text}")
print(f"Sentiment: {result.meta.get('gpt_sentiment', 'Not processed')}")
print(f"Result type: {type(result)}")
print()


In [ ]:
# Test with an actual utterance object
from convokit import Utterance, Speaker

test_utterance = Utterance(
    id="test_utt",
    text="This is terrible! I hate it!",
    speaker=Speaker(id="test_speaker")
)

result = sentiment_transformer.transform_single(test_utterance)
print(f"Input utterance: {test_utterance.text}")
print(f"Speaker: {test_utterance.speaker.id}")
print(f"Sentiment: {result.meta.get('gpt_sentiment', 'Not processed')}")
print()


In [ ]:
# Example: Testing conversation-level transformation
conversation_transformer = LLMPromptTransformer(
    provider="gpt",
    model="gpt-4o-mini",
    object_level="conversation",
    prompt="Summarize this conversation in one sentence: {formatted_object}",
    formatter=lambda conv: " ".join([utt.text for utt in conv.iter_utterances()]),
    metadata_name="conversation_summary",
    config_manager=config
)

test_conv = corpus.get_conversation(corpus.get_conversation_ids()[0])
result = conversation_transformer.transform_single(test_conv)
print(f"Conversation ID: {test_conv.id}")
print(f"Summary: {result.meta.get('conversation_summary', 'Not processed')}")
print()
